In [51]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Цели

### собрать препроцессинг в функцию, т.к. впадлу писать всё заново для тестовых данных
### почему последняя строчка не обрабатывается?
### заменить fit_transform на fit_transorm + transform т.е. разбивать данные на выборки В НАЧАЛЕ



### Pipeline
### Визуализация
### Попробовать модель посложнее?

# Подготовка данных
## Инициализация нужных инструментов

In [42]:
train_data = pd.read_csv('./data/train.csv', index_col = "Id")

my_ord_encoder = OrdinalEncoder()
my_OH_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
my_s_imputer_num = SimpleImputer()
my_s_imputer_cat = SimpleImputer(strategy='most_frequent')

print(train_data.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

## Обработка пропущенных значений

In [46]:
nan_col_list = train_data.columns[train_data.isna().any()].tolist()
nan_df = train_data[nan_col_list]

nan_df_num = nan_df.select_dtypes(exclude='object')
nan_df_cat = nan_df.select_dtypes(include='object')

imputed_nan_df_num = pd.DataFrame(my_s_imputer_num.fit_transform(nan_df_num))
imputed_nan_df_num.columns = nan_df_num.columns

imputed_nan_df_cat = pd.DataFrame(my_s_imputer_cat.fit_transform(nan_df_cat))
imputed_nan_df_cat.columns = nan_df_cat.columns

train_data[imputed_nan_df_cat.columns] = imputed_nan_df_cat
train_data[imputed_nan_df_num.columns] = imputed_nan_df_num

train_data = train_data.drop([1460], axis = 0)
train_data.tail()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1455,20,1.0,85.00000,7500,Pave,Grvl,3.0,3.0,0.0,4.0,...,0,2.000000,2.00000,Shed,0,10,2009,8.0,4.0,185000
1456,60,3.0,66.00000,7917,Pave,Grvl,3.0,3.0,0.0,4.0,...,0,2.000000,0.00000,Shed,0,8,2007,8.0,4.0,175000
1457,20,3.0,68.00000,13175,Pave,Grvl,3.0,3.0,0.0,4.0,...,0,2.000000,2.00000,Shed,0,2,2010,8.0,4.0,210000
1458,70,3.0,75.00000,9042,Pave,Grvl,3.0,3.0,0.0,4.0,...,0,2.000000,2.00000,Shed,2500,5,2010,8.0,4.0,266500
1459,20,3.0,70.05342,9717,Pave,Grvl,3.0,3.0,0.0,4.0,...,0,1.995888,1.88965,Shed,0,4,2010,8.0,4.0,142125


## Перевод данных с категориальными признаками в числовые 
### Те, в которых нет закономерности - с помощью преобразования в признаки с одним активным состоянием
### Те, в которых можно уловить закономерность - в порядковые признаки

In [47]:
obj_df = train_data.select_dtypes(include='object')

OH_col_list = ['Street', 'Alley', 'BldgType', 'Neighborhood', 'RoofStyle', 'RoofMatl',
               'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation','Heating',
              'Electrical','GarageType', 'MiscFeature']
Ord_col_list = ['MSZoning', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
                'LandSlope', 'Condition1', 'Condition2','HouseStyle', 'ExterQual',
               'ExterCond', 'BsmtQual','BsmtCond','BsmtExposure', 'BsmtFinType1', 
                'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual',
               'Functional', 'FireplaceQu','GarageFinish','GarageQual','GarageCond',
               'PavedDrive', 'PoolQC', 'Fence', 'SaleType', 'SaleCondition']

train_data[Ord_col_list] = my_ord_encoder.fit_transform(train_data[Ord_col_list])

OH_cols = pd.DataFrame(my_OH_encoder.fit_transform(train_data[OH_col_list]))
num_cols = train_data.drop(OH_col_list, axis = 1)
OH_cols.index = train_data.index

end_train_data = pd.concat([num_cols, OH_cols], axis = 1)
end_train_data.columns = end_train_data.columns.astype(str)

# Обучение модели

In [80]:
my_model = LinearRegression()
X = end_train_data.drop(["SalePrice"], axis = 1)
y = end_train_data["SalePrice"]

#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.02, random_state = 1)

my_model.fit(X, y);

# preds = my_model.predict(X_valid)
# print(mean_absolute_error(preds, y_valid))